In [1]:
from keras import layers
from keras import models
from keras import backend

class VisionNet:
    model = None
    def __init__ (self,arch,verbose=0,reset=1):
        if (reset):
            print ("INFO: resetting all nodes .. ");
            backend.reset_uids()
        self.model = models.Sequential()
        
        # checks
        if (len(arch) < 2):
            raise Exception('insufficient arguments')
            
        if (arch[0][0] != 'input'):
            raise Exception('architecture has no \'input\' entry')
        
        # parse input
        if (len(arch[0][2]) != 2):
            raise Exception('\'input\' entry is incorrect',arch[0])
            
        input_dep = arch[0][1]
        input_len = arch[0][2][0]
        input_wid = arch[0][2][1]
        print("N/Net: Input len=%d wid=%d depth=%d"%(input_len,input_wid,input_dep))
        
        for (indx,val) in enumerate(arch):
            if (indx == 0):
                continue
            
            if (type(val) == str):
                layer = val
            else:
                layer = val[0]
            
            if (layer == 'conv2d'):
                filters     = val[1]
                kernel_size = (val[2][0],val[2][1])
                act         = val[3]
                print ("CONV::",filters,kernel_size,act)
                
                if (indx==1):
                    self.model.add(layers.Conv2D(filters, kernel_size, activation=act,
                                                 input_shape=(input_len, input_wid, input_dep)))
                else:
                    self.model.add(layers.Conv2D(filters,kernel_size, activation=act))
                                   
            elif (layer == 'maxpool'):
                # default
                pool_size = (2,2)
                print ("MAXP::",pool_size)
                self.model.add(layers.MaxPooling2D(pool_size))
                
            elif (layer == 'flatten'):
                print ("FLAT::")
                self.model.add(layers.Flatten())
                
            elif (layer == 'dense'):
                num_neuron = val[1]
                act        = val[2]
                print ("DENS::",num_neuron,act)
                self.model.add(layers.Dense(num_neuron, activation=act))
            
            else:
                raise Exception('no such layer:',layer)
        
        if (verbose):
            self.model.summary()
    
    def model_summary(self):
        self.model.summary()
    
    def print_model_layer(self,layer):
        for layer in self.model.layers:
            print ("Layer",layer.get_config())

Using TensorFlow backend.
/usr/local/Cellar/python3/3.6.3/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [4]:
inp_len = 28
inp_wid = 28
inp_dep = 3

# number of classifiers i.e. outputs
n_class = 10

# network architecture
NetArch=[
        # input
        ('input',inp_dep,(inp_len,inp_wid)),
    
        # conv + max pool layers
        ('conv2d' ,  32,(3,3),'relu'),
        ('maxpool'),
        ('conv2d' ,  64,(3,3),'relu'),
        ('maxpool'),
        ('conv2d' , 128,(3,3),'relu'),
    
        # flatten + fully connected
        ('flatten'),
        ('dense'  , 64,       'relu'),
        ('dense'  , n_class,  'relu')
      ]
    
my_model_1 = VisionNet(NetArch,verbose=1)

INFO: resetting all nodes .. 
N/Net: Input len=28 wid=28 depth=3
CONV:: 32 (3, 3) relu
MAXP:: (2, 2)
CONV:: 64 (3, 3) relu
MAXP:: (2, 2)
CONV:: 128 (3, 3) relu
FLAT::
DENS:: 64 relu
DENS:: 10 relu
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        896       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 3, 3, 128)         73856     
_________________________________________________________________
flatten_1 (